# Contact-based Object Classification

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [69]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Tuple, Dict
import torch
from torch.utils.data import Dataset, DataLoader
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
from torchsummary import summary
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
import wandb
import torch.nn.functional as F
import hashlib
from typing import Dict, Tuple
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [3]:
# Categories for classification
CATEGORIES = {
    'Blueball': 0,
    'Box': 1,
    'Pencilcase': 2,
    'Pinkball': 3,
    'StuffedAnimal': 4,
    'Tennis': 5,
    'Waterbottle': 6,
}

In [4]:
# Configuration Dictionary
config = {
    'batch_size': 32,
    'lr': 0.001,
    'epochs': 20,
    'data_dir': "/content/drive/MyDrive/IDL/IDL_Data",
    'checkpoint_dir': "/content/drive/MyDrive/IDL/Checkpoint",
    'cleaned_dir': "/content/drive/MyDrive/IDL/IDL_Cleaned",
}

In [67]:
# class ContactDatapointDataset(Dataset):
#     def __init__(self, data_dir: str, labels: dict = None):
#         self.data_dir = Path(data_dir)
#         self.file_paths = list(self.data_dir.glob("*.txt"))
#         self.label_map = labels or {}

#         self.features_list = []
#         self.labels_list = []
#         self.file_indices = []

#         print("Processing files and extracting datapoints:")
#         for file_idx, file_path in enumerate(self.file_paths):
#             try:
#                 category = re.sub(r"\d+", "", file_path.stem)
#                 label = self.label_map.get(category, -1)

#                 if label == -1:
#                     print(f"⚠️ Skipping unknown category: {file_path.name} (parsed: '{category}')")
#                     continue

#                 df = self._parse_file(file_path)

#                 for i in range(len(df)):
#                     features = self._extract_features(df.iloc[i])
#                     self.features_list.append(features)
#                     self.labels_list.append(label)
#                     self.file_indices.append(file_idx)

#                 print(f"  {file_path.name}: {len(df)} datapoints")

#             except Exception as e:
#                 print(f"❌ Error processing {file_path.name}: {e}")

#         self.features = torch.FloatTensor(self.features_list)
#         self.labels = torch.LongTensor(self.labels_list)
#         self.file_indices = torch.LongTensor(self.file_indices)
#         self._print_dataset_stats()

#     def _parse_file(self, file_path: Path) -> pd.DataFrame:
#         df = pd.read_csv(file_path, header=None, skiprows=1)
#         columns = [
#             'timestamp_pc', 'timestamp_micro', 'x', 'y', 'angle_1', 'angle_2',
#             'contact_1_left', 'contact_1_right', 'contact_2_left', 'contact_2_right'
#         ]
#         df = pd.DataFrame(df.values, columns=columns)
#         df = df[df['y'] >= 0].reset_index(drop=True)
#         return df

#     def _extract_features(self, row: pd.Series) -> np.ndarray:
#         return np.array([
#             row['contact_1_left'], row['contact_1_right'],
#             row['contact_2_left'], row['contact_2_right'],
#             row['x'], row['y'], row['angle_1'], row['angle_2']
#         ])

#     def _print_dataset_stats(self):
#         total = len(self.features)
#         unique_files = len(torch.unique(self.file_indices))
#         category_counts = dict(Counter(self.labels_list))
#         print("\nDataset Statistics:")
#         print(f"Total datapoints: {total}")
#         print(f"Total files: {unique_files}")
#         for label, count in category_counts.items():
#             name = [k for k, v in self.label_map.items() if v == label][0]
#             print(f"  {name}: {count} datapoints")

#     def __len__(self):
#         return len(self.features)

#     def __getitem__(self, idx):
#         return self.features[idx], self.labels[idx]

In [99]:
# def split_dataset_by_files_robust(dataset, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, random_state=42):
#     random.seed(random_state)
#     np.random.seed(random_state)

#     assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-10

#     unique_file_indices = torch.unique(dataset.file_indices).numpy()
#     file_to_label = {}
#     for file_idx in unique_file_indices:
#         mask = dataset.file_indices == file_idx
#         window_indices = torch.where(mask)[0]
#         labels = dataset.labels[window_indices]
#         unique_labels, counts = torch.unique(labels, return_counts=True)
#         most_common_label = unique_labels[torch.argmax(counts)].item()
#         file_to_label[file_idx] = most_common_label

#     label_to_files = {}
#     for file_idx, label in file_to_label.items():
#         if label not in label_to_files:
#             label_to_files[label] = []
#         label_to_files[label].append(file_idx)

#     train_files, val_files, test_files = [], [], []
#     for label, files in label_to_files.items():
#         random.shuffle(files)
#         num_files = len(files)

#         if num_files >= 3:
#             num_train = max(1, int(num_files * train_ratio))
#             num_val = max(1, int(num_files * val_ratio))
#             num_test = max(1, num_files - num_train - num_val)

#             if num_train + num_val + num_test > num_files:
#                 if num_files == 2:
#                     num_train, num_val, num_test = 1, 0, 1
#                 else:
#                     num_train = max(1, num_files - 2)
#                     num_val, num_test = 1, 1

#             train_files.extend(files[:num_train])
#             val_files.extend(files[num_train:num_train+num_val])
#             test_files.extend(files[num_train+num_val:])

#         elif num_files == 2:
#             train_files.append(files[0])
#             val_files.append(files[1])

#         elif num_files == 1:
#             train_files.append(files[0])

#     train_indices, val_indices, test_indices = [], [], []
#     for idx in range(len(dataset)):
#         file_idx = dataset.file_indices[idx].item()
#         if file_idx in train_files:
#             train_indices.append(idx)
#         elif file_idx in val_files:
#             val_indices.append(idx)
#         elif file_idx in test_files:
#             test_indices.append(idx)

#     return train_indices, val_indices, test_indices

In [100]:
# def create_dataloaders(data_dir, categories, batch_size=64, seed=42):
#     dataset = ContactDatapointDataset(data_dir=data_dir, labels=categories)
#     train_indices, val_indices, test_indices = split_dataset_by_files_robust(dataset, random_state=seed)

#     train_subset = Subset(dataset, train_indices)
#     train_labels = [dataset.labels[i] for i in train_indices]
#     class_sample_count = torch.tensor([train_labels.count(i) for i in range(len(categories))], dtype=torch.float)
#     weights = 1. / class_sample_count
#     samples_weight = torch.tensor([weights[t] for t in train_labels])
#     sampler = torch.utils.data.WeightedRandomSampler(weights=samples_weight, num_samples=len(samples_weight), replacement=True)

#     train_loader = DataLoader(train_subset, batch_size=batch_size, sampler=sampler)
#     val_loader = DataLoader(Subset(dataset, val_indices), batch_size=batch_size)
#     test_loader = DataLoader(Subset(dataset, test_indices), batch_size=batch_size)

#     print(f"\nFinal dataset splits:")
#     print(f"Train: {len(train_indices)} samples")
#     print(f"Validation: {len(val_indices)} samples")
#     print(f"Test: {len(test_indices)} samples")

#     return train_loader, val_loader, test_loader, weights.to(torch.float32)

In [103]:
# train_loader, val_loader, test_loader, class_weights = create_dataloaders(
#     data_dir=config['data_dir'],
#     categories=CATEGORIES,         # Your label mapping dictionary
#     batch_size=config['batch_size'],
# )


Processing files and extracting datapoints:
  Pinkball2.txt: 3418 datapoints
  Pinkball6.txt: 3396 datapoints
  Blueball6.txt: 3504 datapoints
  Waterbottle2.txt: 3343 datapoints
  Waterbottle4.txt: 4129 datapoints
  Blueball10.txt: 4315 datapoints
  StuffedAnimal7.txt: 3852 datapoints
  Pinkball5.txt: 3547 datapoints
  Waterbottle10.txt: 3888 datapoints
  Tennis7.txt: 3886 datapoints
  Waterbottle8.txt: 3638 datapoints
  Pencilcase5.txt: 4046 datapoints
  Pinkball11.txt: 4054 datapoints
  StuffedAnimal4.txt: 3760 datapoints
  Blueball5.txt: 3512 datapoints
  Pencilcase10.txt: 4014 datapoints
  Tennis4.txt: 4058 datapoints
  Tennis9.txt: 3508 datapoints
  Blueball7.txt: 3613 datapoints
  Tennis1.txt: 3805 datapoints
  Waterbottle7.txt: 3780 datapoints
  Tennis2.txt: 5052 datapoints
  Blueball9.txt: 3498 datapoints
  Tennis6.txt: 3729 datapoints
  Box10.txt: 3373 datapoints
  Tennis3.txt: 3764 datapoints
  Pinkball9.txt: 3389 datapoints
  Blueball11.txt: 3064 datapoints
  Pencilcase3.tx


Final dataset splits:
Train: 183641 samples
Validation: 28674 samples
Test: 56606 samples


In [109]:
#Dataset Creation with window
class ContactWindowDataset(Dataset):
    def __init__(self, data_dir: str, labels: Dict[str, int] = None, window_size: int = 50, step_size: int = 10):
        """
        Args:
            data_dir (str): Directory containing the .txt files
            labels (Dict[str, int]): Dictionary mapping categories to class labels
            window_size (int): Size of the sliding window (smaller = more samples)
            step_size (int): Step size for the sliding window (smaller = more samples)
        """
        self.data_dir = Path(data_dir)
        self.file_paths = list(self.data_dir.glob("*.txt"))
        self.labels = labels or {}
        self.window_size = window_size
        self.step_size = step_size

        # Lists to store all windows and their labels
        self.features_list = []
        self.labels_list = []
        self.file_indices = []  # To keep track of which file each window came from

        # Process all files
        print("Processing files and extracting windows:")
        for file_idx, file_path in enumerate(self.file_paths):
            try:
                category = re.sub(r"\d+", "", file_path.stem)
                label = self.labels.get(category, -1)

                if label == -1:
                    print(f"⚠️ Skipping file with unknown category: {file_path.name} (parsed category: '{category}')")
                    continue  # Skip this file entirely

                df = self._parse_file(file_path)

                windows_from_file = 0
                for start_idx in range(0, len(df) - self.window_size + 1, self.step_size):
                    window = df.iloc[start_idx:start_idx + self.window_size]
                    features = self._extract_features(window)
                    self.features_list.append(features)
                    self.labels_list.append(label)
                    self.file_indices.append(file_idx)
                    windows_from_file += 1

                print(f"  {file_path.name}: {windows_from_file} windows generated from {len(df)} datapoints")

            except Exception as e:
                print(f"❌ Error processing file: {file_path.name}")
                print(f"   Error: {e}")

        # Convert lists to tensors for efficiency
        self.features = torch.FloatTensor(self.features_list)
        self.labels = torch.LongTensor(self.labels_list)
        self.file_indices = torch.LongTensor(self.file_indices)

        # Print summary statistics
        self._print_dataset_stats()

    def _parse_file(self, file_path: Path) -> pd.DataFrame:
        """Parse a single data file and filter out rows where y < 0"""
        df = pd.read_csv(file_path, header=None, skiprows=1)
        columns = [
            'timestamp_pc', 'timestamp_micro',
            'x', 'y', 'angle_1', 'angle_2',
            'contact_1_left', 'contact_1_right',
            'contact_2_left', 'contact_2_right'
        ]
        df = pd.DataFrame(df.values, columns=columns)

        # Filter out rows where y < 0
        df = df[df['y'] >= 0].reset_index(drop=True)
        return df

    def _extract_features(self, window: pd.DataFrame) -> np.ndarray:
        """Extract features from a window of data"""
        # Basic statistical features
        features = np.array([
            window['contact_1_left'].mean(),
            window['contact_1_right'].mean(),
            window['contact_2_left'].mean(),
            window['contact_2_right'].mean(),
            window['x'].max() - window['x'].min(),
            window['y'].max() - window['y'].min(),
            window['angle_1'].std(),
            window['angle_2'].std(),
            # Additional features for more information
            window['contact_1_left'].std(),
            window['contact_1_right'].std(),
            window['contact_2_left'].std(),
            window['contact_2_right'].std(),
            window['angle_1'].mean(),
            window['angle_2'].mean()
        ])
        return features

    def _print_dataset_stats(self):
        """Print statistics about the dataset"""
        total_windows = len(self.features)
        unique_files = len(torch.unique(self.file_indices))

        # Count samples per category
        category_counts = {}
        for label in self.labels_list:
            category_name = list(CATEGORIES.keys())[list(CATEGORIES.values()).index(label)]
            category_counts[category_name] = category_counts.get(category_name, 0) + 1

        print("\nDataset Statistics:")
        print(f"Total number of windows: {total_windows}")
        print(f"Total number of files: {unique_files}")
        print(f"Average windows per file: {total_windows / unique_files:.2f}")
        print("\nSamples per category:")
        for category, count in category_counts.items():
            print(f"  {category}: {count} windows")

    def __len__(self) -> int:
        return len(self.features)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, int]:
        return self.features[idx], self.labels[idx]

In [110]:
def split_dataset_by_files_robust(dataset, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, random_state=42):
    """
    Split the dataset by files while trying to ensure all classes are represented in each split.
    This function handles edge cases where some classes have very few examples.

    Parameters:
    -----------
    dataset : ContactWindowDataset
        The dataset to split
    train_ratio : float
        Ratio of data for training
    val_ratio : float
        Ratio of data for validation
    test_ratio : float
        Ratio of data for testing
    random_state : int
        Random seed for reproducibility

    Returns:
    --------
    tuple
        (train_indices, val_indices, test_indices)
    """
    import numpy as np
    import random

    # Set random seed for reproducibility
    random.seed(random_state)
    np.random.seed(random_state)

    # Verify ratios sum to 1
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-10, "Ratios must sum to 1"

    # Get unique files and their labels
    unique_file_indices = torch.unique(dataset.file_indices).numpy()

    # Create mapping from file index to most common label in that file
    file_to_label = {}
    for file_idx in unique_file_indices:
        # Get all windows from this file
        mask = dataset.file_indices == file_idx
        window_indices = torch.where(mask)[0]

        # Get the labels for these windows
        labels = dataset.labels[window_indices]

        # Use the most common label for this file
        unique_labels, counts = torch.unique(labels, return_counts=True)
        most_common_label = unique_labels[torch.argmax(counts)].item()
        file_to_label[file_idx] = most_common_label

    # Group files by label
    label_to_files = {}
    for file_idx, label in file_to_label.items():
        if label not in label_to_files:
            label_to_files[label] = []
        label_to_files[label].append(file_idx)

    # Print how many files per label
    print("\nFiles per label:")
    for label, files in label_to_files.items():
        class_name = list(CATEGORIES.keys())[list(CATEGORIES.values()).index(label)]
        print(f"  {class_name}: {len(files)} files")

    # Manual split to ensure all classes are in all splits
    train_files = []
    val_files = []
    test_files = []

    for label, files in label_to_files.items():
        # Shuffle files for this label
        random.shuffle(files)

        # Calculate how many files should go to each split
        num_files = len(files)

        if num_files >= 3:
            # If we have enough files, use the ratios
            num_train = max(1, int(num_files * train_ratio))
            num_val = max(1, int(num_files * val_ratio))
            num_test = max(1, num_files - num_train - num_val)

            # If we don't have enough files for all splits, prioritize train
            if num_train + num_val + num_test > num_files:
                # If we have 2 files, put one in train and one in test
                if num_files == 2:
                    num_train, num_val, num_test = 1, 0, 1
                else:
                    num_train = max(1, num_files - 2)
                    num_val = 1
                    num_test = 1

            # Split files
            train_files.extend(files[:num_train])
            val_files.extend(files[num_train:num_train+num_val])
            test_files.extend(files[num_train+num_val:])

        elif num_files == 2:
            # For classes with only 2 files, put one in train and one in val
            train_files.append(files[0])
            val_files.append(files[1])
            # We'll handle test set separately

        elif num_files == 1:
            # For classes with only 1 file, duplicate windows
            # Put the file in the train set
            train_files.append(files[0])
            print(f"Warning: Class {label} has only one file, putting it in train set")

    # Now map back to window indices
    train_indices = []
    val_indices = []
    test_indices = []

    for idx in range(len(dataset)):
        file_idx = dataset.file_indices[idx].item()
        if file_idx in train_files:
            train_indices.append(idx)
        elif file_idx in val_files:
            val_indices.append(idx)
        elif file_idx in test_files:
            test_indices.append(idx)

    # Special handling for classes with only one or two files
    # For these classes, create synthetic test examples
    for label, files in label_to_files.items():
        if len(files) <= 2:
            class_indices = []
            for idx in range(len(dataset)):
                if dataset.labels[idx].item() == label:
                    class_indices.append(idx)

            if len(files) == 2 and len(test_indices) == 0:
                # If we have 2 files but none in test, move some val windows to test
                class_val_indices = [idx for idx in class_indices if idx in val_indices]
                if class_val_indices:
                    # Move half of val windows to test
                    half_point = len(class_val_indices) // 2
                    for idx in class_val_indices[half_point:]:
                        val_indices.remove(idx)
                        test_indices.append(idx)

            elif len(files) == 1:
                # If we have only 1 file, duplicate some train windows to val and test
                class_train_indices = [idx for idx in class_indices if idx in train_indices]

                # Select random indices to duplicate (without removing from train)
                random.shuffle(class_train_indices)
                third_point = max(1, len(class_train_indices) // 3)

                # Add to val and test
                val_indices.extend(class_train_indices[:third_point])
                test_indices.extend(class_train_indices[third_point:2*third_point])

    # Print class distribution in each split
    print("\nClass distribution:")
    for split_name, indices in [
        ("Train", train_indices),
        ("Validation", val_indices),
        ("Test", test_indices)
    ]:
        class_counts = {}
        for idx in indices:
            label = dataset.labels[idx].item()
            class_name = list(CATEGORIES.keys())[list(CATEGORIES.values()).index(label)]
            class_counts[class_name] = class_counts.get(class_name, 0) + 1

        print(f"\n{split_name} set:")
        for class_name, count in sorted(class_counts.items()):
            print(f"  {class_name}: {count} windows")

    return train_indices, val_indices, test_indices

In [111]:
# Create the dataset with smaller window and step size to maximize samples
window_size = 100   # Smaller window size generates more samples
step_size = 5     # Smaller step size creates more overlapping windows
dataset = ContactWindowDataset(
    data_dir=config["data_dir"],
    labels=CATEGORIES,
    window_size=window_size,
    step_size=step_size
)

Processing files and extracting windows:
  Pinkball2.txt: 664 windows generated from 3418 datapoints
  Pinkball6.txt: 660 windows generated from 3396 datapoints
  Blueball6.txt: 681 windows generated from 3504 datapoints
  Waterbottle2.txt: 649 windows generated from 3343 datapoints
  Waterbottle4.txt: 806 windows generated from 4129 datapoints
  Blueball10.txt: 844 windows generated from 4315 datapoints
  StuffedAnimal7.txt: 751 windows generated from 3852 datapoints
  Pinkball5.txt: 690 windows generated from 3547 datapoints
  Waterbottle10.txt: 758 windows generated from 3888 datapoints
  Tennis7.txt: 758 windows generated from 3886 datapoints
  Waterbottle8.txt: 708 windows generated from 3638 datapoints
  Pencilcase5.txt: 790 windows generated from 4046 datapoints
  Pinkball11.txt: 791 windows generated from 4054 datapoints
  StuffedAnimal4.txt: 733 windows generated from 3760 datapoints
  Blueball5.txt: 683 windows generated from 3512 datapoints
  Pencilcase10.txt: 783 windows ge

In [112]:
# Split the dataset by files to prevent data leakage
train_indices, val_indices, test_indices = split_dataset_by_files_robust(dataset)

# Create subset datasets
from torch.utils.data import Subset
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['batch_size'])
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'])

# Print split information
print("\nDataset Splits:")
print(f"Training set size: {len(train_dataset)} windows")
print(f"Validation set size: {len(val_dataset)} windows")
print(f"Test set size: {len(test_dataset)} windows")

# Example of accessing a batch
for features, labels in train_loader:
    print(f"\nSample batch:")
    print(f"Batch features shape: {features.shape}")
    print(f"Batch labels shape: {labels.shape}")
    break


Files per label:
  Pinkball: 11 files
  Blueball: 11 files
  Waterbottle: 10 files
  StuffedAnimal: 10 files
  Tennis: 10 files
  Pencilcase: 9 files
  Box: 10 files

Class distribution:

Train set:
  Blueball: 4883 windows
  Box: 5085 windows
  Pencilcase: 4710 windows
  Pinkball: 4901 windows
  StuffedAnimal: 5481 windows
  Tennis: 5360 windows
  Waterbottle: 5376 windows

Validation set:
  Blueball: 660 windows
  Box: 674 windows
  Pencilcase: 935 windows
  Pinkball: 658 windows
  StuffedAnimal: 973 windows
  Tennis: 991 windows
  Waterbottle: 708 windows

Test set:
  Blueball: 2008 windows
  Box: 1448 windows
  Pencilcase: 1391 windows
  Pinkball: 1981 windows
  StuffedAnimal: 1411 windows
  Tennis: 1412 windows
  Waterbottle: 1361 windows

Dataset Splits:
Training set size: 35796 windows
Validation set size: 5599 windows
Test set size: 11012 windows

Sample batch:
Batch features shape: torch.Size([32, 14])
Batch labels shape: torch.Size([32])


In [114]:
class Temporal1DBlock(nn.Module):
    """Residual block for 1D time series data with skip connection"""
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, dilation=1):
        super(Temporal1DBlock, self).__init__()
        padding = dilation * (kernel_size - 1) // 2

        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, dilation)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, stride=1, padding=padding, dilation=dilation)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += identity
        return self.relu(out)


class TimeSeriesClassifier(nn.Module):
    def __init__(self, input_features, num_classes, window_size=50):
        super(TimeSeriesClassifier, self).__init__()

        self.window_size = window_size

        self.feature_embedding = nn.Sequential(
            nn.Linear(input_features, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True)
        )

        self.conv1 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size=7, stride=1, padding=3),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2, padding=1)  # 128-dim
        )

        self.res1 = Temporal1DBlock(128, 128, dilation=1)
        self.pool1 = nn.MaxPool1d(2)

        self.res2 = Temporal1DBlock(128, 256, dilation=2)
        self.pool2 = nn.MaxPool1d(2)

        self.res3 = Temporal1DBlock(256, 512, dilation=4)

        self.global_pool = nn.AdaptiveAvgPool1d(1)

        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(512, 128)
        self.bn = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        if len(x.shape) == 2:
            x = x.unsqueeze(-1)
            x = x.transpose(1, 2)
            x = x.view(x.shape[0], -1)
            x = self.feature_embedding(x)
            x = x.unsqueeze(-1)
            x = x.repeat(1, 1, self.window_size)
        elif len(x.shape) == 3:
            batch_size, features, time_steps = x.shape
            x = x.transpose(1, 2).reshape(-1, features)
            x = self.feature_embedding(x)
            x = x.view(batch_size, time_steps, -1).transpose(1, 2)

        x = self.conv1(x)

        x = self.res1(x)
        x = self.pool1(x)

        x = self.res2(x)
        x = self.pool2(x)

        x = self.res3(x)

        x = self.global_pool(x).squeeze(-1)

        x = self.dropout(x)
        x = self.fc1(x)
        x = self.bn(x)
        feats = x
        out = self.fc2(x)

        return {"feats": feats, "out": out}


# Initialize the model
input_features = 14  # Number of features in your dataset
window_size = 50  # Size of time window
num_classes = len(CATEGORIES)
model = TimeSeriesClassifier(input_features, num_classes, window_size)
model = model.to(device)

summary(model, input_size=(input_features, window_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             960
       BatchNorm1d-2                   [-1, 64]             128
              ReLU-3                   [-1, 64]               0
            Conv1d-4              [-1, 128, 50]          57,472
       BatchNorm1d-5              [-1, 128, 50]             256
              ReLU-6              [-1, 128, 50]               0
         MaxPool1d-7              [-1, 128, 25]               0
            Conv1d-8              [-1, 128, 25]          49,280
       BatchNorm1d-9              [-1, 128, 25]             256
             ReLU-10              [-1, 128, 25]               0
           Conv1d-11              [-1, 128, 25]          49,280
      BatchNorm1d-12              [-1, 128, 25]             256
             ReLU-13              [-1, 128, 25]               0
  Temporal1DBlock-14              [-1, 

In [107]:
# class Temporal1DBlock(nn.Module):
#     def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, dilation=1):
#         super(Temporal1DBlock, self).__init__()
#         padding = dilation * (kernel_size - 1) // 2

#         self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding, dilation)
#         self.bn1 = nn.BatchNorm1d(out_channels)
#         self.relu = nn.ReLU(inplace=True)

#         self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, stride=1, padding=padding, dilation=dilation)
#         self.bn2 = nn.BatchNorm1d(out_channels)

#         self.downsample = nn.Sequential()
#         if in_channels != out_channels:
#             self.downsample = nn.Sequential(
#                 nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride),
#                 nn.BatchNorm1d(out_channels)
#             )

#     def forward(self, x):
#         identity = self.downsample(x)
#         out = self.relu(self.bn1(self.conv1(x)))
#         out = self.bn2(self.conv2(out))
#         out += identity
#         return self.relu(out)

# class TimeSeriesClassifier(nn.Module):
#     def __init__(self, input_features, num_classes, simulated_timesteps=10):
#         super(TimeSeriesClassifier, self).__init__()

#         reduced_dim = 64
#         self.simulated_timesteps = simulated_timesteps

#         self.feature_embedding = nn.Sequential(
#             nn.Linear(input_features, reduced_dim),
#             nn.ReLU(inplace=True)
#         )

#         self.conv1 = nn.Sequential(
#             nn.Conv1d(reduced_dim, 128, kernel_size=3, padding=1),
#             nn.BatchNorm1d(128),
#             nn.ReLU(inplace=True),
#             nn.MaxPool1d(kernel_size=2)
#         )

#         self.res_blocks = nn.Sequential(
#             Temporal1DBlock(128, 256, dilation=1),
#             Temporal1DBlock(256, 512, dilation=2),
#             Temporal1DBlock(512, 1024, dilation=4)
#         )

#         self.global_pool = nn.AdaptiveAvgPool1d(1)
#         self.classifier = nn.Sequential(
#             nn.Dropout(0.5),
#             nn.Linear(1024, num_classes)
#         )

#     def forward(self, x):
#         if len(x.shape) == 2:
#             x = self.feature_embedding(x)
#             x = torch.nan_to_num(x)  # Replace NaNs
#             x = x.unsqueeze(-1).repeat(1, 1, self.simulated_timesteps)
#         elif len(x.shape) == 3:
#             batch_size, features, time_steps = x.shape
#             x = x.transpose(1, 2).reshape(-1, features)
#             x = self.feature_embedding(x)
#             x = torch.nan_to_num(x)  # Replace NaNs
#             x = x.view(batch_size, time_steps, -1).transpose(1, 2)

#         x = self.conv1(x)
#         x = self.res_blocks(x)
#         x = self.global_pool(x).squeeze(-1)
#         out = self.classifier(x)
#         return {"feats": x, "out": out}

# # Initialize the model
# input_features = 8  # Number of features in your dataset
# window_size = 50  # Size of time window
# num_classes = len(CATEGORIES)
# model = TimeSeriesClassifier(input_features, num_classes, window_size)
# model = model.to(device)

# summary(model, input_size=(input_features, window_size))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             576
              ReLU-2                   [-1, 64]               0
            Conv1d-3              [-1, 128, 50]          24,704
       BatchNorm1d-4              [-1, 128, 50]             256
              ReLU-5              [-1, 128, 50]               0
         MaxPool1d-6              [-1, 128, 25]               0
            Conv1d-7              [-1, 256, 25]          33,024
       BatchNorm1d-8              [-1, 256, 25]             512
            Conv1d-9              [-1, 256, 25]          98,560
      BatchNorm1d-10              [-1, 256, 25]             512
             ReLU-11              [-1, 256, 25]               0
           Conv1d-12              [-1, 256, 25]         196,864
      BatchNorm1d-13              [-1, 256, 25]             512
             ReLU-14              [-1, 

In [115]:
class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [116]:
def accuracy(output, target, topk=(1,)):
    """Computes the top-k accuracy"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [117]:
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    # Metric meters
    loss_m = AverageMeter()
    acc_m = AverageMeter()

    # Progress Bar
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()  # Zero gradients

        # Get the data
        x, y = data
        x, y = x.to(device), y.to(device)

        # Forward pass
        outputs = model(x)
        loss = criterion(outputs['out'], y)

        # Backward and optimize
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        optimizer.step()

        # Calculate accuracy
        acc = accuracy(outputs['out'], y)[0].item()

        # Update meters
        loss_m.update(loss.item())
        acc_m.update(acc)

        # Update progress bar
        batch_bar.set_postfix(
            loss="{:.04f}".format(float(loss_m.avg)),
            acc="{:.04f}%".format(float(acc_m.avg)),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr']))
        )
        batch_bar.update()

        # Memory management
        del x, y, outputs, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    return loss_m.avg, acc_m.avg

In [118]:
@torch.no_grad()
def validate_model(model, val_loader, criterion, class_names, device):
    model.eval()
    # Metric meters
    loss_m = AverageMeter()
    acc_m = AverageMeter()

    # Progress Bar
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    all_preds = []
    all_targets = []

    for i, data in enumerate(val_loader):
        x, y = data
        x, y = x.to(device), y.to(device)

        # Forward pass
        outputs = model(x)
        loss = criterion(outputs['out'], y)

        # Calculate accuracy
        acc = accuracy(outputs['out'], y)[0].item()

        # Store predictions and targets for confusion matrix
        _, predicted = torch.max(outputs['out'], 1)
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(y.cpu().numpy())

        # Update meters
        loss_m.update(loss.item())
        acc_m.update(acc)

        # Update progress bar
        batch_bar.set_postfix(
            loss="{:.04f}".format(float(loss_m.avg)),
            acc="{:.04f}%".format(float(acc_m.avg))
        )
        batch_bar.update()

        # Memory management
        del x, y, outputs, loss
        torch.cuda.empty_cache()


    batch_bar.close()

    # Calculate per-class accuracy
    if class_names:
        print("\nPer-class Validation Accuracy:")
        per_class_acc = {}
        for i, class_name in enumerate(class_names):
            class_mask = (np.array(all_targets) == i)
            if np.sum(class_mask) > 0:
                class_correct = np.sum((np.array(all_preds)[class_mask] == i))
                class_total = np.sum(class_mask)
                acc_percent = 100 * class_correct / class_total
                print(f"  {class_name}: {acc_percent:.4f}% ({class_correct}/{class_total})")
                per_class_acc[f"val_acc_{class_name}"] = acc_percent


    return loss_m.avg, acc_m.avg

In [119]:
def save_model(model, optimizer, scheduler, metrics, epoch, path):
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
        'metrics': metrics
    }, path)

In [120]:
# Define CrossEntropyLoss as the criterion
criterion = nn.CrossEntropyLoss(
    weight=class_weights.to(device),
    label_smoothing=0.1
)

# Initialize optimizer with AdamW
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config['lr'],
    weight_decay=1e-5
)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=True
)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [121]:
wandb.login(key="78d5988d9f05a421bc74d044c3cd9afc3b918020") # API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [122]:
# Initialize wandb
run = wandb.init(
    name = "14run", ## Wandb creates random run names if you skip this field
    reinit = False, ### Allows reinitalizing runs when you re-run this cell
    #id = "", ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "object_classification", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

In [ ]:
# Create checkpoint directory if it doesn't exist
os.makedirs(config['checkpoint_dir'], exist_ok=True)

# Initialize best metrics tracking
best_val_loss = float('inf')
best_val_acc = 0
class_names = list(CATEGORIES.keys())

# Training loop
for epoch in range(config['epochs']):
    print(f"\nEpoch {epoch + 1}/{config['epochs']}")

    # Training phase
    train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")

    # Validation phase
    val_loss, val_acc = validate_model(model, val_loader, criterion, class_names, device)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.2f}%")

    # Update learning rate scheduler
    scheduler.step(val_loss)
    curr_lr = optimizer.param_groups[0]['lr']

    # Save best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_val_acc = val_acc

        # Save best model
        best_model_path = os.path.join(config['checkpoint_dir'], 'best_model.pth')
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': val_loss,
            'val_acc': val_acc,
        }, best_model_path)
        wandb.save(best_model_path)  # Save the model to WandB
        print(f"Saved best model with validation loss: {best_val_loss:.4f} and accuracy: {best_val_acc:.2f}%")

    # Save the model for every epoch
    last_model_path = os.path.join(config['checkpoint_dir'], f'model_epoch_{epoch+1}.pth')
    torch.save(model.state_dict(), last_model_path)
    wandb.save(last_model_path)  # Save the model to WandB
    print(f"Saved model for epoch {epoch+1}")

    # Logging metrics to WandB
    wandb.log({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'train_acc': train_acc,
        'val_loss': val_loss,
        'val_acc': val_acc,
        'learning_rate': curr_lr
    }, step=epoch)

    print(f"End of Epoch {epoch+1}/{config['epochs']}")

# Final message
print(f"\nTraining complete! Best validation accuracy: {best_val_acc:.2f}%")


Epoch 1/20


Train Loss: 1.9171, Train Accuracy: 20.77%



Per-class Validation Accuracy:
  Blueball: 48.7879% (322/660)
  Box: 42.8783% (289/674)
  Pencilcase: 28.7701% (269/935)
  Pinkball: 28.1155% (185/658)
  StuffedAnimal: 11.0997% (108/973)
  Tennis: 18.8698% (187/991)
  Waterbottle: 0.0000% (0/708)
Validation Loss: 2.0207, Validation Accuracy: 24.29%
Saved best model with validation loss: 2.0207 and accuracy: 24.29%
Saved model for epoch 1
End of Epoch 1/20

Epoch 2/20


Train:  48%|████▊     | 542/1119 [00:09<00:11, 50.57it/s, acc=23.8893%, loss=1.8718, lr=0.001000]wandb: WARNING Tried to log to step 0 that is less than the current step 8. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.


Train Loss: 1.8674, Train Accuracy: 24.02%


...

Train Loss: 1.6370, Train Accuracy: 36.61%



Per-class Validation Accuracy:
  Blueball: 27.5758% (182/660)
  Box: 56.8249% (383/674)
  Pencilcase: 37.0053% (346/935)
  Pinkball: 25.0760% (165/658)
  StuffedAnimal: 19.8356% (193/973)
  Tennis: 27.2452% (270/991)
  Waterbottle: 5.9322% (42/708)
Validation Loss: 2.1802, Validation Accuracy: 28.24%
Saved model for epoch 17
End of Epoch 17/20

Epoch 18/20


Train Loss: 1.6326, Train Accuracy: 37.15%



Per-class Validation Accuracy:
  Blueball: 25.7576% (170/660)
  Box: 58.1602% (392/674)
  Pencilcase: 37.5401% (351/935)
  Pinkball: 27.9635% (184/658)
  StuffedAnimal: 17.3690% (169/973)
  Tennis: 23.4107% (232/991)
  Waterbottle: 5.0847% (36/708)
Validation Loss: 2.1803, Validation Accuracy: 27.40%
Saved model for epoch 18
End of Epoch 18/20

Epoch 19/20


Train:  34%|███▎      | 376/1119 [00:07<00:12, 57.57it/s, acc=37.4834%, loss=1.6227, lr=0.000250]

In [59]:
def test_model(model, test_loader, criterion, class_names, device, checkpoint_dir=None):
    """
    Evaluate the model on the test dataset and generate detailed performance metrics
    """
    # Set model to evaluation mode
    model.eval()
    # Initialize metrics
    test_loss = 0.0
    correct = 0
    total = 0
    # Store all predictions and ground truth for analysis
    all_preds = []
    all_targets = []
    all_probs = [] # Store probabilities for confidence analysis
    # Per-class statistics
    class_correct = {class_name: 0 for class_name in class_names}
    class_total = {class_name: 0 for class_name in class_names}
    # Create progress bar
    test_bar = tqdm(test_loader, desc="Testing", unit="batch", ncols=100)

    with torch.no_grad():
        for data in test_bar:
            # Get inputs and labels
            inputs, targets = data
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)

            # Handle different output formats (dictionary vs. tensor)
            if isinstance(outputs, dict) and 'out' in outputs:
                outputs_for_loss = outputs['out']
            else:
                outputs_for_loss = outputs

            loss = criterion(outputs_for_loss, targets)

            # Calculate loss and accuracy
            test_loss += loss.item() * inputs.size(0)

            # Get predictions and probabilities
            if isinstance(outputs, dict) and 'out' in outputs:
                probs = torch.nn.functional.softmax(outputs['out'], dim=1)
                _, predicted = torch.max(outputs['out'], 1)
            else:
                probs = torch.nn.functional.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)

            # Update total counts
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

            # Update per-class counts
            for i in range(targets.size(0)):
                label = targets[i].item()
                pred = predicted[i].item()
                class_name = class_names[label]
                class_total[class_name] += 1
                if pred == label:
                    class_correct[class_name] += 1

            # Store predictions and targets for later analysis
            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

            # Update progress bar
            test_bar.set_postfix({
                'loss': f"{test_loss/total:.4f}",
                'acc': f"{100.0*correct/total:.2f}%"
            })

    # Calculate overall metrics
    test_loss /= len(test_loader.dataset)
    test_acc = correct / total

    # Calculate per-class accuracy
    class_accuracy = {name: class_correct[name]/class_total[name] if class_total[name] > 0 else 0
                    for name in class_names}

    # Print results
    print("\n" + "="*50)
    print("TEST RESULTS")
    print("="*50)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc:.4f} ({correct}/{total})")
    print("\nPer-Class Accuracy:")
    for class_name in class_names:
        print(f" {class_name}: {class_accuracy[class_name]:.4f} ({class_correct[class_name]}/{class_total[class_name]})")

    # Return comprehensive metrics dictionary
    return {
        'test_loss': test_loss,
        'test_accuracy': test_acc,
        'class_accuracy': class_accuracy,
        'predictions': all_preds,
        'targets': all_targets,
        'probabilities': all_probs
    }

In [60]:
# Load the best model (optional - if you saved a checkpoint)
best_model_path = f"{config['checkpoint_dir']}/best_model.pth"
if os.path.exists(best_model_path):
    checkpoint = torch.load(best_model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Loaded best model from epoch {checkpoint.get('epoch', 'unknown')}")

Loaded best model from epoch 10


In [61]:

# Evaluate on test set
test_results = test_model(
    model=model,
    test_loader=test_loader,
    criterion=criterion,
    class_names=class_names,
    device=device,
    checkpoint_dir=config['checkpoint_dir']
)


Testing: 100%|████████████████████████| 345/345 [00:03<00:00, 88.81batch/s, loss=2.4440, acc=15.35%]



TEST RESULTS
Test Loss: 2.4440
Test Accuracy: 0.1535 (1690/11012)

Per-Class Accuracy:
 Blueball: 0.0946 (190/2008)
 Box: 0.1678 (243/1448)
 Pencilcase: 0.1359 (189/1391)
 Pinkball: 0.1807 (358/1981)
 StuffedAnimal: 0.1446 (204/1411)
 Tennis: 0.2677 (378/1412)
 Waterbottle: 0.0940 (128/1361)


In [62]:
run.finish()

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
learning_rate,██████▄▄▄▄▄▄▃▃▃▃▃▃▂▁
train_acc,▁▂▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇█
train_loss,█▇▆▆▆▅▅▅▄▄▄▄▄▃▃▃▃▂▂▁
val_acc,▁▅▆▆▅▆▆▆▆▆▇█▇█▇██▇▇▇
val_loss,▁▃▆▅▇▃▄▆▅▅▅▆▅▆▇▆▆▆▅█
epoch,20
learning_rate,2e-05
train_acc,45.55686
train_loss,1.47727
val_acc,27.07661
